# 2DCOS Analysis Toolkit for CD
Perform complete Two-Dimensional Correlation Spectroscopy (2DCOS) analysis  
on Far-UV Circular Dichroism (CD) data, including MRE conversion, preprocessing,  
correlation map computation, peak detection, and result export.

In [ ]:
#@title **Setup (Colab)**
#@markdown Prepare the environment for analysis in Google Colab. Run once before starting.

import os
import sys
import subprocess
from pathlib import Path

repo_url = "https://github.com/hubertstanczak/2dcos_toolkit.git"
repo_name = "2dcos_toolkit"

try:
    os.chdir("/content")

    if not Path(repo_name).exists():
        print(f"🔄 Cloning repository '{repo_name}'...")
        subprocess.check_call(["git", "clone", repo_url, repo_name])

    os.chdir(repo_name)

    if not (Path.cwd() / "pyproject.toml").exists():
        raise FileNotFoundError("Setup failed: 'pyproject.toml' not found in the repository root.")

    print("⚙️ Installing backend (editable)...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", "."])

    from dcos_toolbox.api import (
        init_session,
        load_input_data_and_parse,
        compute_mre,
        compute_2dcos,
        visualize_session,
        package_results,
    )

    print("✅ Setup ready.")

except subprocess.CalledProcessError as e:
    print(f"❌ Setup error: command failed (exit code {e.returncode}).")
    raise
except Exception as e:
    print(f"❌ Setup error: {e}")
    raise

In [ ]:
#@title **Input Data (Colab)**
#@markdown Upload one or more CD data files in **.csv**, **.xlsx**, **.xls** or a **.zip** archive containing such files.
#@markdown
#@markdown **Expected table format:**
#@markdown - **Row 1 (header):** perturbation values for columns 2..N
#@markdown - **Column 1:** wavelength **λ (nm)**
#@markdown - **Body:** CD values (**mdeg**)
#@markdown
#@markdown **JASCO:** ASCII/CSV exports are supported; the tool reads the CD signal from **Channel 1**.

from pathlib import Path
from google.colab import files  #type: ignore

job_name = "my_analysis"  #@param {type:"string"}

input_dir  = Path("data/input_cd")
output_dir = Path("data/results")

SESSION = init_session(job_name=job_name, output_dir=str(output_dir))
SESSION.input_dir = input_dir

SESSION.input_dir.mkdir(parents=True, exist_ok=True)
SESSION.output_dir.mkdir(parents=True, exist_ok=True)

print("Input dir :", SESSION.input_dir.resolve())
print("Output dir:", SESSION.output_dir.resolve())

print("📤 Upload CD files (.csv/.xlsx/.xls) or a .zip")
uploaded = files.upload()
for name, data in uploaded.items():
    (SESSION.input_dir / name).write_bytes(data)

load_input_data_and_parse(SESSION, paths=[SESSION.input_dir])
print("✅ Input parsing done.")

In [ ]:
#@title CD parameters for MRE calculation
#@markdown ## Formula for Mean Residue Ellipticity
#@markdown $$[\theta] = \frac{M \cdot \theta_{\text{obs}}}{10 \cdot c \cdot l \cdot n}$$
#@markdown **Where:**
#@markdown - **M** – molar mass of the peptide *(g/mol)*
#@markdown - **θ_obs** – observed ellipticity *(mdeg)*
#@markdown - **c** – concentration of the sample *(mg/mL)*
#@markdown - **l** – optical path length *(cm)*
#@markdown - **n** – number of peptide bonds *(≈ number of residues)*
#@markdown ---
#@markdown ### Parameter description
#@markdown All uploaded CD files are assumed to correspond to the **same sample**.
#@markdown The parameters below (**n, c, l, M**) are applied to **all uploaded datasets**.
#@markdown For different peptides with different physical parameters,
#@markdown please run the notebook separately for each peptide.
#@markdown
#@markdown - `residues_number` (**n**) – number of peptide bonds / residues
#@markdown - `concentration` (**c**) – peptide concentration in **mg/mL**
#@markdown - `path_length` (**l**) – cuvette optical path length in **cm**
#@markdown - `molar_mass` (**M**) – peptide molar mass in **g/mol**

molar_mass     = 10   #@param {type:"number"}
concentration  = 10   #@param {type:"number"}
path_length    = 10   #@param {type:"number"}
residues_number = 22  #@param {type:"number"}

#@markdown ---
#@markdown ### Plot generator
#@markdown If enabled, 1D spectra will be shown and saved for **all** datasets.
generate_mre_plot = True   #@param {type:"boolean"}
use_mre_for_plot  = False  #@param {type:"boolean"}
#@markdown - If `True`, plots show calculated **MRE** [θ].
#@markdown - If `False`, plots show **original CD** in mdeg.



compute_mre(
    SESSION,
    residues_number=residues_number,
    concentration_mg_ml=concentration,
    path_length_mm=path_length,
    molar_mass_g_mol=molar_mass,
    generate_plot=generate_mre_plot,
    use_mre_for_plot=use_mre_for_plot,
)
print("✅ MRE calculation done.")

In [ ]:
#@title **2D-COS Calculation**
#@markdown Compute **synchronous (Φ)** and **asynchronous (Ψ)** 2D correlation maps for all parsed datasets.
#@markdown ---
#@markdown **Input signal**
use_mre_for_2dcos = False  #@param {type:"boolean"}
#@markdown - If `True`, use **MRE** (requires the MRE step to be completed first).
#@markdown - If `False`, use **raw CD**.

#@markdown ---
#@markdown **Reference spectrum (subtraction mode)**
reference_type = "mean"  #@param ["mean", "first", "last", "none"]
#@markdown - **mean**: subtract the mean spectrum (recommended default)
#@markdown - **first**: subtract the first spectrum
#@markdown - **last**: subtract the last spectrum
#@markdown - **none**: no subtraction (uses the original spectra)


compute_2dcos(
    SESSION,
    use_mre_for_2dcos=use_mre_for_2dcos,
    reference_type=reference_type,
)
print("✅ 2DCOS calculation done.")

In [ ]:
#@title **2D-COS Visualization**
#@markdown Generate high-resolution **combined figures** (Φ + Ψ) with optional peak annotations.
#@markdown ---
#@markdown ### Plot settings
colormap = "jet"  #@param ["jet", "bwr", "coolwarm", "viridis", "seismic"]
lambda_min = 0.0      #@param {type:"number"}
lambda_max = 0.0      #@param {type:"number"}
#@markdown - If `lambda_min >= lambda_max`, the full wavelength range is used.

#@markdown ---
#@markdown ### Peak mirroring
mark_mirror_peaks = True  #@param {type:"boolean"}
#@markdown - If `True`, peaks are marked on both symmetric halves of the map (mirrored across the diagonal).
#@markdown - If `False`, peaks are marked only on one half; mirrored positions are not shown.

#@markdown ---
#@markdown ### Peak annotation (how many peaks to label)
#@markdown Select the number of most prominent peaks in each category (0–5). Use small values for readability.
#@markdown
#@markdown **Synchronous Φ**
n_sync_diag_peaks = "0"          #@param ["0","1","2","3","4","5"]
#@markdown - Diagonal (auto-peaks)

n_sync_cross_max_peaks = "0"     #@param ["0","1","2","3","4","5"]
#@markdown - Positive cross-peaks (Φ > 0)

n_sync_cross_min_peaks = "0"     #@param ["0","1","2","3","4","5"]
#@markdown - Negative cross-peaks (Φ < 0)

#@markdown **Asynchronous Ψ**
n_async_cross_max_peaks = "0"    #@param ["0","1","2","3","4","5"]
#@markdown - Positive cross-peaks (Ψ > 0)

n_async_cross_min_peaks = "0"    #@param ["0","1","2","3","4","5"]
#@markdown - Negative cross-peaks (Ψ < 0)


visualize_session(
    SESSION,
    colormap=colormap,
    lambda_min=lambda_min,
    lambda_max=lambda_max,
    mark_mirror_peaks=mark_mirror_peaks,
    n_sync_diag_peaks=n_sync_diag_peaks,
    n_sync_cross_max_peaks=n_sync_cross_max_peaks,
    n_sync_cross_min_peaks=n_sync_cross_min_peaks,
    n_async_cross_max_peaks=n_async_cross_max_peaks,
    n_async_cross_min_peaks=n_async_cross_min_peaks,
)
print("✅ Visualization done.")

In [ ]:
#@title **Export Results**
#@markdown Package selected files into a ZIP file (and download it in Colab).

include_input_file     = True  #@param {type:"boolean"}
#@markdown Include the original input CD files.

include_mre            = True  #@param {type:"boolean"}
#@markdown - Include MRE tables.

include_mre_plot       = True  #@param {type:"boolean"}
#@markdown - Include 1D MRE plots.

include_2dcos          = True  #@param {type:"boolean"}
#@markdown - Include 2D-COS matrices.

include_2dcos_plot     = True  #@param {type:"boolean"}
#@markdown - Include 2D-COS map plots.

try:
    zip_path = package_results(
        SESSION,
        include_input_file=include_input_file,
        include_mre=include_mre,
        include_mre_plot=include_mre_plot,
        include_2dcos=include_2dcos,
        include_2dcos_plot=include_2dcos_plot,
    )
    files.download(str(zip_path))
except Exception as e:
    print(e)

## How to use this notebook 

### 1) Workflow (run top → bottom)
1. **Setup** — installs the backend and imports the API.
2. **Input Data** — upload your files using the **popup file picker** (`.csv`, `.xls`, `.xlsx` or `.zip`).  
   Expected table: **col 1 = wavelength (nm)**, **cols 2..N = spectra values (mdeg)**, header = temperatures/perturbation values.  
   **JASCO files are supported**: the parser detects `XYDATA` and reads the CD signal from **Channel 1**.
3. **MRE** — enable if you want to convert CD → MRE (provide residues, concentration, path length, molar mass).
4. **2D-COS** — computes **Φ (synchronous)** and **Ψ (asynchronous)** maps (requires ≥ 3 spectra per dataset).
5. **Visualization** — generates combined high-resolution figures (optional peak labels).
6. **Export ZIP** — creates one ZIP with results (one folder per input dataset) and triggers a **download**.

### 2) What outputs you get
For each dataset you may obtain:
- `*_sync.csv` (Φ), `*_async.csv` (Ψ)
- `*_2DCOS_combined.png` (combined figure)
- optional: `*_MRE.csv`, `*_CD_plot.png` / `*_MRE_plot.png`
- final: `<job_name>_2DCOS_results.zip`

### 3) Quick interpretation (Φ/Ψ)
- **Φ (synchronous)**: shows which wavelengths change **together** during the perturbation.  
  - Φ > 0: same-direction changes; Φ < 0: opposite-direction changes.  
  - Diagonal peaks mark wavelengths with the strongest overall change.
- **Ψ (asynchronous)**: highlights **sequential / delayed** relationships (which region responds earlier vs later).  
  - Ψ is antisymmetric and the diagonal is ~0.

**Order of changes (Noda’s rule, temperature increasing):**  
For a pair (λ1, λ2), check the signs at the same coordinate:
- If **Φ and Ψ have the same sign** → change at **λ1 occurs before λ2**  
- If **Φ and Ψ have opposite signs** → change at **λ2 occurs before λ1**
